In [1]:
%load_ext autoreload
%autoreload 2

from optimizers.perturbed_gd import PerturbedGD
from optimizers.cubic_reg import StochasticCubicRegularizedNewton
from models.nn import Net

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.nn import MSELoss
import torch.nn.functional as F
import torch.autograd as autograd
from copy import deepcopy


In [4]:

model = Net(3, 1)
loss_fn = MSELoss()

input = torch.ones(3)
target = torch.tensor(.8)

optimizer = PerturbedGD(model.parameters(),
                        l=1,
                        rho=1,
                        epsilon=1e-4,
                        c=.5,
                        delta=.1,
                        delta_f=10,
                        d=2)
optimizer.zero_grad()
loss_fn(model(input), target).backward()
optimizer.step()


Adding noise


KeyboardInterrupt: 

In [ ]:

grad_batch_size = 64
hess_batch_size = 64
model = Net()
loss_fn = F.nll_loss

optimizer = StochasticCubicRegularizedNewton(model.parameters(),
                                             l=10,
                                             rho=10,
                                             epsilon=1e-4)

train_loader_grad = torch.utils.data.DataLoader(
    MNIST('data/',
          train=True,
          download=True,
          transform=Compose([
            ToTensor(),
            Normalize((0.1307,), (0.3081,))
          ])),
    batch_size=grad_batch_size, shuffle=True)

train_loader_hess = torch.utils.data.DataLoader(
    MNIST('data/',
          train=True,
          download=True,
          transform=Compose([
            ToTensor(),
            Normalize((0.1307,), (0.3081,))
          ])),
    batch_size=hess_batch_size, shuffle=True)

iter_grad = iter(train_loader_grad)
iter_hess = iter(train_loader_hess)
batch_idx = 0
while True:
    optimizer.zero_grad()
    
    data_for_grad = next(iter_grad, None)
    data_for_hess = next(iter_hess, None)
    
    if data_for_grad is None or data_for_hess is None:
        print("Exhausted training data -- finished optimization")
        break
    features_g, labels_g = data_for_grad
    features_h, labels_h = data_for_hess
    
    loss_h = loss_fn(model(features_h), labels_h)
    grad_loss_h, = autograd.grad(loss_h, model.parameters())
    
    loss_g = loss_fn(model(features_g), labels_g)
    loss_g.backward()
    
    optimizer.step(grad_loss_h)
    
    batch_idx += 1
    
    
for batch_idx, (data, labels) in enumerate(train_loader_grad):
    
    optimizer.zero_grad()
    
    loss = loss_fn(model(data), labels)
    loss.backward()
    
    optimizer.step()
